In [7]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    # if "2.Final_output" in list_of_folders:
    #     subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    # os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [58]:
import os
from Bio import SeqIO
import subprocess

family_group_list = ["3.Satyrine","3.Satyrine_Exon5_split","3.Satyrine_Exon5_split_for_dup","4.Pierinae","5.Coliadinae","6.Heliconiinae_Danainae_Nymphalinae","7.Papilionidae","8.Lycaenidae"]
for family_group in family_group_list:
    output_location = f"/mnt/g/My Drive/Circadian Rhythm Genes Project/10.Cycle_Exon_Analysis/{family_group}"
    make_output_folder(output_location)
    genome_location = "/mnt/f/Genomes_2023-12-26"
    list_of_species = os.listdir(f"{output_location}/1.Blast_result")
    if 'desktop.ini' in list_of_species:
        list_of_species.remove("desktop.ini")
    # print(list_of_species)
    # list_of_species = ["Heliconius_charithonia"]
    # list_of_species = ["Maniola_hyperantus"]
    error_exons = ''
    for species in list_of_species:
        print(species)
        genome_file = get_genome_file(genome_location,species)
        print(genome_file)
        previous_coordinate_start = ''
        previous_scaffold = ''
        with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
            coordinate_file_list = coordinate_final_file.readlines()
        
        for i in range(1,len(coordinate_file_list)):
            
            coordinate_line_split = coordinate_file_list[i].split(",")
            
                
    #         print(coodinate_line_split[5])
            if coordinate_line_split[5] == "N":
                if previous_coordinate_start == '':
                    previous_coordinate_start = coordinate_line_split[2]
                    previous_scaffold = coordinate_line_split[1]
                complement = coordinate_line_split[4]
                start,stop = coordinate_line_split[2],coordinate_line_split[3]
                print(start,stop)
                query_name = coordinate_line_split[6]
                scaffold = coordinate_line_split[1]
                exon = f"Exon_{query_name.split('_')[-1]}"
                left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
                print(left_oh, right_oh)
    #             print(complement)
                if complement == "0":
                    if start < previous_coordinate_start or scaffold != previous_scaffold:
                        print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                        prompt = input("Continue (Y/N)?")
                        if prompt[0].lower() == "n":
                            assert False
                elif complement == "1":
                    if start > previous_coordinate_start or scaffold != previous_scaffold:
                        print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                        prompt = input("Continue (Y/N)?")
                        if prompt[0].lower() == "n":
                            assert False
                else:
                    print("ERROR")
                
            elif coordinate_line_split[5] == "Y":
                query_name = coordinate_line_split[6]
                exon = f"Exon_{query_name.split('_')[-1]}"
                scaffold = coordinate_line_split[1]
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                continue
                
            gene_sequence = get_gene_sequence(genome_location, 
                                              species, 
                                              genome_file, 
                                              scaffold,
                                              start,
                                              stop,
                                              complement,
                                              output_location)
            
            gene_sequence_2 = get_gene_sequence(genome_location, 
                                              species, 
                                              genome_file, 
                                              scaffold,
                                              int(start)-2,
                                              int(stop)+2,
                                              complement,
                                              output_location)
        
            current_exon = query_name.split('_')[-1]
            if current_exon != "1":        
                if gene_sequence_2[:2].lower() != "ag":
                    print(species, current_exon)
                    print(gene_sequence)
                    print(gene_sequence_2)
                    print("Not AG")
                    test = input("quit?")
                    if test == "y":
                        assert False
                    # assert False
            print(current_exon, gene_sequence[-3:].translate())
            if gene_sequence[-3:].translate() != "*":
                
                # print(gene_sequence_2[-2:].lower())
                if gene_sequence_2[-2:].lower() != "gt":
                    print(current_exon)
                    print(len(coordinate_file_list))
                    print(species, current_exon)
                    print(gene_sequence)
                    print(gene_sequence_2)
                    print("Not gt")
                    test = input("quit?")
                    if test == "y":
                        assert False
            if current_exon == "1":
                if complement == 1:
                    translated = gene_sequence.reverse_complement.translate()
                else:
                    translated = gene_sequence.translate()
                print(translated)
                if "*" in translated:
                    error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                else:
                    exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                    print(exon_output)
            
                    write_exon_output(output_location,species,exon,exon_output)
            elif current_exon == "16":
                if complement == 1:
                    translated = gene_sequence[:-int(left_oh)].reverse_complement.translate()
                    print(gene_sequence)
                else:
                    translated = gene_sequence[int(left_oh):].translate()
                    print(gene_sequence)
    
                if "*" in translated[:-1]:
                    error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                    print(translated)
                else:
                    
                    exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                    print(exon_output)
                    write_exon_output(output_location,species,exon,exon_output)
    #         assert False
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
    
                write_exon_output(output_location,species,exon,exon_output)
            
    #         assert False
            previous_coordinate_start = start
            previous_scaffold = scaffold
        error_exons += "\n\n"
    #     print(coordinate_file_list)
    #     break
    coordinate_file = ''
    print(error_exons)
    with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
        error_out_file.write(error_exons)

Erebia_aethiops
GCA_923060345.2_ilEreAeth2.2_genomic.fna
2287647 2287741
0 2
Getting Gene
Getting Gene
1 A
MARDFARVHEYYLQLHELQPPPSHFTCQYDL
>Erebia_aethiops_Exon_1_OV281079.1_2287647_2287741_left_0_right_2
ATGGCGCGCGATTTCGCGCGAGTTCACGAGTATTACCTCCAACTACACGAATTGCAGCCGCCGCCGTCGCACTTTACCTGCCAGTATGACCTGCA
2260903 2261076
1 2
Getting Gene
Getting Gene
2 H
>Erebia_aethiops_Exon_2_OV281079.1_2260903_2261076_left_1_right_2
GTCGCACGCGAACGGCGCGCCGGCGTACGAGCTGACCATGGGCGCGGGCGTCGGGGACGCGAGCGCCGCGGGCTGCGCGGAGAGCGCCGGGGGCACGCTCGTgcacgcgccgcacgccgcCCACCCGCCGCCGCAGCCGCACGTCCCGCGCGCTCGGAAGAACTCCTACACCAT
2260031 2260150
1 2
Getting Gene
Getting Gene
3 M
>Erebia_aethiops_Exon_3_OV281079.1_2260031_2260150_left_1_right_2
GCGCGGCGCATGTTCGGAGGCGGACAGCGAGGGTTGCTACCCCACAGAGCTCCCGGAACACCCTTACGCGTGCCAACAGCAAGCCTCGCAACCTATGCGCAAACGCACCAACTTACAATG
2248797 2248868
1 2
Getting Gene
Getting Gene
4 K
>Erebia_aethiops_Exon_4_OV281079.1_2248797_2248868_left_1_right_2
TTGCACAAGCAGTACATACGACGATGATGGCAGTGAGGATAGTCGATCCGTCCGC

quit? 


2 F
>Argynnis_bischoffii_washingtonia_Exon_2_CM066637.1_9914196_9914303_left_1_right_2
GCTGACGACGGGCGTGGAGGGCGTCCCGCCCTGCGACGCCATATCGATGCTTAGTGCGCCACACAATCATCATGTCAACCTCAACCCTAGGAAACGGAAGAGCCCTTT
9912032 9912133
1 2
Getting Gene
Getting Gene
3 F
>Argynnis_bischoffii_washingtonia_Exon_3_CM066637.1_9912032_9912133_left_1_right_2
CTCGGACGGCTACGATGTTCCTAGCTGTGACGTGCCGCTGTCACAGCCCCCAACGTCCGTGCAAGAACCAGCAAGTGCTACGCGAAAAAGAAAGACGTCTTC
9904092 9904163
1 2
Getting Gene
Getting Gene
4 K
>Argynnis_bischoffii_washingtonia_Exon_4_CM066637.1_9904092_9904163_left_1_right_2
TGGCGGGGGTAGTGCATATGACGACGATGGCAGTGAAGACACCCGGTCCAGCCGGACTTTGCCTGacaaaaa
9901659 9901995
1 0
Getting Gene
Getting Gene
5 Q
>Argynnis_bischoffii_washingtonia_Exon_5_CM066637.1_9901659_9901995_left_1_right_0
ACAGAACCACAGCGAAATTGAAAAACGTCGACGAGATAAAATGAACACATACATATCTGAGTTGAGCGCGATAGTGCCGATGTGCGGTGCAATGGCGCGGAAGCTAGACAAACTGACAGTACTACGAATGGCGGTACAGCATCTTAGGTCTATCCGTGGGGCTCTCTCTGCCGGCCCATTGACAGTTAGACCCAGACCATCCTTCCTACCAGAGAGAGAGTTAAACGCC

quit? 


2 L
>Dryadula_phaetusa_Exon_2_JAOYRS010000001.1_1317326_1317433_left_1_right_2
GCTGACAACGGGCGTGGAGGGCGTCCCGCCCTGCGACGCCCGCTCTATGCTCACAGCGCCGCACGCGCACCACGTGACAATCAACCCCAGGAAACGAAAGAGCCCCTA
1315065 1315169
1 2
Getting Gene
Getting Gene
3 L
>Dryadula_phaetusa_Exon_3_JAOYRS010000001.1_1315065_1315169_left_1_right_2
ttctgACAGCTATGAAGTGTCAAGCTGTGACGTGCCACTGTCGCGGCCGCCAGCCCCAGTACAGGAACCGGCCTGTGCGACTAGGAAAAGGAAGGTGTCGACCTA
1308612 1308683
1 2
Getting Gene
Getting Gene
4 E
>Dryadula_phaetusa_Exon_4_JAOYRS010000001.1_1308612_1308683_left_1_right_2
tgGAACTGGCAGTGCATACGATGATGACGGCAGCGAGGACGCGCGTTCAAGTCGCACTCTTCCCGACAAGAA
1307758 1308094
1 0
Getting Gene
Getting Gene
5 Q
>Dryadula_phaetusa_Exon_5_JAOYRS010000001.1_1307758_1308094_left_1_right_0
ACAAAACCACAGCGAGATTGAGAAACGTAGGCGAGATAAGATGAACACATATATATCTGAGTTAAGCGCGATAGTACCAATGTGTGGGGCGATGGCCAGGAAGTTGGACAAGCTGACAGTACTCCGAATGGCTGTGCAGCATTTGCGCTCAATCCGAGGGGCCCTCTCCGCTGGTCCGCTTACAGTCAGGCCAAGGCCCTCTTTCCTGTCCGAAAGAGAGCTAAACGCCCTCGTGCTGCAAGCGGCTCGCGATTGTTT

...

2 L
>Heliconius_nattereri_Exon_2_JAAIXI010000253.1_657212_657319_left_1_right_2
GCTGACGACGGGTGCTGAGGGCGTCCCGCCCTGTGACACCCGCTCGATGCTCACGGCGCCGCACGCGCACCACGTGACACTTAACCCGAGGAAACGGAAGAGCCCCTT
655686 655790
1 2
Getting Gene
Getting Gene
3 L
>Heliconius_nattereri_Exon_3_JAAIXI010000253.1_655686_655790_left_1_right_2
cTCAGACGGTTATGACGTAGCCAGCTGCGATGTTCCCCTATCGCGGCCGCCCGCCCCCGCGCAGGAAGCTGCCAGCGCCACTAGGAAAAGAAAGGCATCTACCTA
651401 651472
1 2
Getting Gene
Getting Gene
4 K
>Heliconius_nattereri_Exon_4_JAAIXI010000253.1_651401_651472_left_1_right_2
tGGAGCCGGCAGTGCCTACGACGACGATGGCAGTGAAGATGCGCGCTCTAGCCGCACTCTACCCGACAAAAA
649837 650173
1 0
Getting Gene
Getting Gene
5 Q
>Heliconius_nattereri_Exon_5_JAAIXI010000253.1_649837_650173_left_1_right_0
GCAAAACCACAGCGAAATTGAAAAACGTAGACGAGATAAAATGAACACCTATATATCCGAACTAAGCGCGATAGTACCAATGTGTGGAGCGATGGCCAGAAAACTAGACAAACTGACAGTCCTCCGTATGGCAGTTCAGCACTTACGATCGATCAGAGGGGCCCTGTCAGCGGGCCCTCTCACTGCAAGGCCGAGGCCCTCGTTCCTGTCAGAAAGAGAGTTAAACGCCCTCGTCCTGCAAGCGGCTCGCGATTGCTTTC

quit? 


2 L
>Heliconius_sara_Exon_2_OU860800.1_1253735_1253842_left_1_right_2
GCTGACGACGGGTGCTGAGGGCGTCCCGCCCTGTGACACCCGCTCGATGCTCACGGCGCCGCACGCGCACCACGTGACCCTTAACCCGAGGAAACGGAAGAGCCCCTT
1251481 1251585
1 2
Getting Gene
Getting Gene
3 L
>Heliconius_sara_Exon_3_OU860800.1_1251481_1251585_left_1_right_2
CTCAGACGGTTATGACGTAACCAGCTGCGATGTTCCCCTGTCGCGGCCGCCAGCCCCCGCGCAAGAGCCGGCCAGCGCCACTAGGAAAAGAAAGGCATCTACCTA
1246176 1246247
1 2
Getting Gene
Getting Gene
4 K
>Heliconius_sara_Exon_4_OU860800.1_1246176_1246247_left_1_right_2
TGGAGCTGGTAGTGCCTACGATGACGATGGCAGTGAAGATGCGCGCTCTAGCCGCACTCTACCcgacaaaaa
1244624 1244960
1 0
Getting Gene
Getting Gene
5 Q
>Heliconius_sara_Exon_5_OU860800.1_1244624_1244960_left_1_right_0
acAAAATCACAGCGAAATTGAAAAACGTAGACGAGATAAAATGAACACCTATATATCTGAATTGAGCGCAATAGTGCCAATGTGTGGAGCGATGGCCAGGAAGCTGGACAAACTGACAGTCCTCCGTATGGCCGTACAGCACTTACGATCGATCCGAGGGGCCCTGTCAGCTGGCCCCCTCACTGTCAGGCCGAGGCCCTCGTTCCTGTCAGAAAGAGAGCTTAACGCCCTCGTCCTGCAAGCGGCTCGCGATTGCTTTCTGATGGTCGTGGGTTGCGATCGGGGGAGGCTGCT

quit? 


2 L
>Philaethria_dido_Exon_2_JAKGSA010000003.1_333761_333865_left_1_right_2
GCTGACGACGGGCGCGGAGGGCGTCCCGCCCTGCGACGCCCGCTCCATGCTCACGGCGCCGCACGCGCACCACGTGACACTCAACCCCCGGAAACGGAAGCCCTA
331671 331772
1 2
Getting Gene
Getting Gene
3 L
>Philaethria_dido_Exon_3_JAKGSA010000003.1_331671_331772_left_1_right_2
ttcggATGGTTATGACGTAGCCAGCTGTGATGCACTCTCGCGACCGCCGGGCCCCGTCCAGGAGGCGGCTAGCTGTACTAAGAAAAGAAAACTTTCTACtta
323700 323771
1 2
Getting Gene
Getting Gene
4 E
>Philaethria_dido_Exon_4_JAKGSA010000003.1_323700_323771_left_1_right_2
TGGAGCTAGTAGTGCATATGATGATGATGGCAGCGAGGATGCGCGCTCTAGCCGCACTCTACCCGACAAGAA
322462 322798
1 0
Getting Gene
Getting Gene
5 K
>Philaethria_dido_Exon_5_JAKGSA010000003.1_322462_322798_left_1_right_0
ACAAAACCACAGCGAGATTGAAAAGCGTAGACGAGATAAAATGAACACCTATATATCAGAGCTAAGCGCGATTGTGCCAATGTGTGGGGCGATGGCAAGAAAATTGGACAAACTAACAGTCCTTCGTATGGCAGTACAGCATTTGCGGTCAATTCGAGGGGCCCTCTCGGCTGGACCCCTCACTGTCAGGCCAAGGCCCTCGTTCCTGTCAGAGAGGGAGCTAAACGCCCTAGTGCTACAAGCGGCTCGCGATTGCTTCTTGATGGTGGTGGGTTGCGATCG